# Bag-of-words


The bag-of-words model is a simplifying representation used in natural language processing and information retrieval (IR). In this model, a text (such as a sentence or a document) is represented as the bag (multiset) of its words, disregarding grammar and even word order but keeping multiplicity. 

In document classification, a bag of words is a sparse vector of occurrence counts of words; that is, a sparse histogram over the vocabulary.

(https://en.wikipedia.org/wiki/Bag-of-words_model)

In [1]:
from __future__ import unicode_literals 

## Collecting a dataset from ecommerce with product name and category

In [2]:
import gzip
import json

In [3]:
# load json ecommerce dump
corpus = list()
with gzip.open('ecommerce.json.gz') as fp:
    for line in fp:
        entry = line.decode('utf8')
        corpus.append(json.loads(entry))

In [4]:
from pprint import pprint
pprint(corpus[0])

{u'_id': 120008322,
 u'cat': u' Automotivo',
 u'descr': u'Chegou o kit que junta resist\xeancia e conforto, al\xe9m de n\xedveis m\xe1ximos de seguran\xe7a. S\xe3o 4 pneus para seu carro ficar completo e com a qualifica\xe7\xe3o que voc\xea precisa.\nCom os conhecimentos avan\xe7ados de hoje e um entusiasmo pela dire\xe7\xe3o, os engenheiros da Pirelli puderam dar grandes passos. Cada pneu da Pirelli \xe9 respons\xe1vel n\xe3o s\xf3 pelo desempenho, mas tamb\xe9m por uma "vontade de ir pra estrada", comunicando-se com o motorista e gerando um melhor entendimento do desempenho do ve\xedculo, ou seja, a Pirelli transforma a sua viagem em uma aventura divertida e livre de problemas. Pneu Pirelli para carros com rodas aro 16, modelo high performance Phanthon, perfil baixo proporcionando maior estabilidade nas curvas, excelente qualidade e durabilidade para pistas.\n\nImagens meramente ilustrativas.\nTodas as informa\xe7\xf5es divulgadas s\xe3o de responsabilidade do fabricante/fornecedor.'

In [5]:
print corpus[0]['descr']

Chegou o kit que junta resistência e conforto, além de níveis máximos de segurança. São 4 pneus para seu carro ficar completo e com a qualificação que você precisa.
Com os conhecimentos avançados de hoje e um entusiasmo pela direção, os engenheiros da Pirelli puderam dar grandes passos. Cada pneu da Pirelli é responsável não só pelo desempenho, mas também por uma "vontade de ir pra estrada", comunicando-se com o motorista e gerando um melhor entendimento do desempenho do veículo, ou seja, a Pirelli transforma a sua viagem em uma aventura divertida e livre de problemas. Pneu Pirelli para carros com rodas aro 16, modelo high performance Phanthon, perfil baixo proporcionando maior estabilidade nas curvas, excelente qualidade e durabilidade para pistas.

Imagens meramente ilustrativas.
Todas as informações divulgadas são de responsabilidade do fabricante/fornecedor.


In [6]:
import gensim
print gensim.summarization.summarize(corpus[0]['descr'])

São 4 pneus para seu carro ficar completo e com a qualificação que você precisa.


In [7]:
len(corpus)

65875

In [8]:
# let's build a classifier for product categories
# for speed up the example lets only consider the first 10k products
dataset = list()
for entry in corpus[:10000]:
    if 'cat' in entry:
        dataset.append( (entry['name'], entry['cat'].lower().strip()) )

In [9]:
len(dataset)

9953

In [10]:
pprint(dataset[:10])

[(u'Kit com 4 Pneus de Alta Performance Pirelli Aro 16 205/55R16 Phantom',
  u'automotivo'),
 (u'Chandon Brut Ros\xe9 750 ml', u'alimentos e bebidas'),
 (u'Kit com 2 Vodkas Sueca Absolut Vanilia 1000ml', u'alimentos e bebidas'),
 (u'Kit  - Livros de Colorir: Jardim Secreto + Floresta Encantada + Reino Animal',
  u'livros'),
 (u"Livro - Assassin's Creed: Submundo", u'livros'),
 (u'BCAA 2400 - 100 C\xe1psulas - Nitech Nutrition',
  u'suplementos e vitaminas'),
 (u'100% Whey - 900g - Baunilha - Nitech Nutrition',
  u'suplementos e vitaminas'),
 (u'Whey Protein Isolate - 900g - Morango - Nitech Nutrition',
  u'suplementos e vitaminas'),
 (u'100% Whey - 900g - Chocolate - Nitech Nutrition',
  u'suplementos e vitaminas'),
 (u'BCAA 2400 - 200 C\xe1psulas - Nitech Nutrition',
  u'suplementos e vitaminas')]


In [11]:
# how many distinc categories do we have and how many items per category?
from collections import Counter
counter = Counter([cat for prod, cat in dataset])

pprint(counter.most_common())

[(u'beb\xeas', 1208),
 (u'eletroport\xe1teis', 1052),
 (u'automotivo', 915),
 (u'utilidades dom\xe9sticas', 857),
 (u'suplementos e vitaminas', 787),
 (u'ar-condicionado e aquecedores', 754),
 (u'inform\xe1tica', 706),
 (u'cama, mesa e banho', 670),
 (u'tv e home theater', 644),
 (u'perfumaria', 532),
 (u'beleza e sa\xfade', 497),
 (u'dvds e blu-ray', 433),
 (u'rel\xf3gios', 410),
 (u'pet shop', 391),
 (u'instrumentos musicais', 44),
 (u'celulares e telefones', 18),
 (u'eletrodom\xe9sticos', 16),
 (u'\xe1udio', 13),
 (u'livros', 2),
 (u'alimentos e bebidas', 2),
 (u'brinquedos', 1),
 (u'linha industrial', 1)]


# Building a SVM Classifier with bag-of-words

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
import nltk
stopwords = nltk.corpus.stopwords.words('portuguese')

In [14]:
classifier = Pipeline([('vect', TfidfVectorizer()), ('clf', SVC(kernel='linear', probability=True))])
encoder = LabelEncoder()
# Please check on http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [15]:
data = [prod for prod, cat in dataset]
labels = [cat for prod, cat in dataset]
len(data)

9953

In [16]:
target = encoder.fit_transform(labels)

In [17]:
encoder.classes_.item(1)

u'ar-condicionado e aquecedores'

In [18]:
classifier.fit(data, target)

Pipeline(steps=[(u'vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [19]:
classifier.predict(["Refrigerador Brastemp com função frostfree"])

array([9])

In [20]:
print encoder.classes_[9]

eletrodomésticos


In [21]:
probs = classifier.predict_proba(["Ventilador"])

In [22]:
guess = [( class_, probs.item(n)) for n, class_ in enumerate(encoder.classes_)]
pprint(guess)

[(u'alimentos e bebidas', 3.688477530894003e-07),
 (u'ar-condicionado e aquecedores', 2.469095180707653e-07),
 (u'automotivo', 2.245439791949051e-07),
 (u'beb\xeas', 4.366071253426756e-07),
 (u'beleza e sa\xfade', 2.391938864502423e-07),
 (u'brinquedos', 1.4201684720393886e-05),
 (u'cama, mesa e banho', 2.3369519957920863e-07),
 (u'celulares e telefones', 1.2226580639746691e-05),
 (u'dvds e blu-ray', 2.6750925987771174e-07),
 (u'eletrodom\xe9sticos', 8.090091427177432e-07),
 (u'eletroport\xe1teis', 0.9999450205737752),
 (u'inform\xe1tica', 2.7277448331803544e-07),
 (u'instrumentos musicais', 7.482958503364135e-07),
 (u'linha industrial', 1.655290879491242e-05),
 (u'livros', 6.493134239212444e-07),
 (u'perfumaria', 2.3547348349555312e-07),
 (u'pet shop', 3.607951820250581e-07),
 (u'rel\xf3gios', 1.3834263784016502e-06),
 (u'suplementos e vitaminas', 5.505094406822638e-07),
 (u'tv e home theater', 3.421098159233175e-07),
 (u'utilidades dom\xe9sticas', 2.1383913180936887e-07),
 (u'\xe1udi

In [23]:
from operator import itemgetter
for cat, proba in sorted(guess, key=itemgetter(1), reverse=True):
    print '{}: {:.4f}'.format(cat,proba)

eletroportáteis: 0.9999
linha industrial: 0.0000
brinquedos: 0.0000
celulares e telefones: 0.0000
áudio: 0.0000
relógios: 0.0000
eletrodomésticos: 0.0000
instrumentos musicais: 0.0000
livros: 0.0000
suplementos e vitaminas: 0.0000
bebês: 0.0000
alimentos e bebidas: 0.0000
pet shop: 0.0000
tv e home theater: 0.0000
informática: 0.0000
dvds e blu-ray: 0.0000
ar-condicionado e aquecedores: 0.0000
beleza e saúde: 0.0000
perfumaria: 0.0000
cama, mesa e banho: 0.0000
automotivo: 0.0000
utilidades domésticas: 0.0000


# Activity: How to improve the result above?

What causes the result above?
How can we improve?